In [ ]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from operator import itemgetter
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.decomposition import NMF
import scipy
from scipy import linalg
from numpy import dot

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Kaggle PRML/Dataset/train.csv')
songs = pd.read_csv('/content/drive/MyDrive/Kaggle PRML/Dataset/songs.csv')
sfl = pd.read_csv('/content/drive/MyDrive/Kaggle PRML/Dataset/save_for_later.csv')
song_labels = pd.read_csv('/content/drive/MyDrive/Kaggle PRML/Dataset/song_labels.csv')
test = pd.read_csv('/content/drive/MyDrive/Kaggle PRML/Dataset/test.csv')

In [ ]:
customer_dict = {}
for i,cust in enumerate(train["customer_id"].unique()):
  customer_dict[cust] = i

In [ ]:
cust_avg = train.groupby("customer_id")["score"].mean()
song_avg = train.groupby("song_id")["score"].mean()

In [ ]:
scores = np.zeros((14053, 10001))
for index,row in train.iterrows():
  s_id = int(row["song_id"])
  c_id = int(customer_dict[row["customer_id"]])
  scores[c_id][s_id] = row["score"]

lang_scores = np.zeros((14053, 10001))


In [ ]:
X = scores.copy()
latent_features = 500
max_iter=50
error_limit=0.5
fit_error_limit=0.5

eps = 1e-5
print('Starting NMF decomposition with {} latent features and {} iterations.'.format(latent_features, max_iter))

# mask
mask = np.sign(X)

# initial matrices. A is random [0,1] and Y is A\X.
rows, columns = X.shape
np.random.seed(0)
A = np.random.rand(rows, latent_features)
A = np.maximum(A, eps)

Y = linalg.lstsq(A, X)[0]
Y = np.maximum(Y, eps)

masked_X = mask * X
X_est_prev = dot(A, Y)
for i in range(1, max_iter + 1):

  # ===== updates =====
  # Matlab: A=A.*(((W.*X)*Y')./((W.*(A*Y))*Y'));
  top = dot(masked_X, Y.T)
  bottom = (dot((mask * dot(A, Y)), Y.T)) + eps
  A *= top / bottom

  A = np.maximum(A, eps)
  # print 'A',  np.round(A, 2)

  # Matlab: Y=Y.*((A'*(W.*X))./(A'*(W.*(A*Y))));
  top = dot(A.T, masked_X)
  bottom = dot(A.T, mask * dot(A, Y)) + eps
  Y *= top / bottom
  Y = np.maximum(Y, eps)
  # print 'Y', np.round(Y, 2)


  # ==== evaluation ====
  if i % 5 == 0 or i == 1 or i == max_iter:
    print('Iteration {}:'.format(i),)
    X_est = dot(A, Y)
    err = mask * (X_est_prev - X_est)
    fit_residual = np.sqrt(np.sum(err ** 2))
    X_est_prev = X_est

    curRes = linalg.norm(mask * (X - X_est), ord='fro')
    print('fit residual', np.round(fit_residual, 4),)
    print('total residual', np.round(curRes, 4))
    if curRes < error_limit or fit_residual < fit_error_limit:
        break

In [ ]:
new_scores1 = np.dot(A,Y)

In [ ]:
test_scores = []
ca = []
sa = []
for index,row in test.iterrows():
  s_id = int(row["song_id"])
  c_id = int(customer_dict[row["customer_id"]])
  test_scores.append(min(5,new_scores[c_id][s_id]))
  ca.append(cust_avg[row["customer_id"]])
  sa.append(song_avg[s_id])

In [ ]:
dummy = {"score" : test_scores}
predictions = pd.DataFrame(dummy, columns = ['score'])
predictions.index.name = "test_row_id"

In [ ]:
predictions.to_csv("/content/drive/MyDrive/Kaggle PRML/Dataset/test_file.csv")

In [ ]:
dummy = {"score" : test_scores, "ca" : ca, "sa" : sa}
predictions = pd.DataFrame(dummy, columns = ['score','ca','sa'])
predictions.index.name = "test_row_id"

In [ ]:
s_label = [0]*10001
max_count = [0]*10001
for index,row in song_labels.iterrows():
  try:
    ind = platform_dict[row["platform_id"]]
    if max_count[ind] < int(row["count"]):
      max_count[ind] = int(row["count"])
      s_label[ind] = int(row["label_id"])
      break
  except:
    pass

In [ ]:
songs['released_year'].isna().sum()

16